In [30]:
import json
import math
import numpy as np
import random
import sklearn
import string
from collections import defaultdict
from nltk.stem.porter import *
from sklearn import linear_model
from gensim.models import Word2Vec
import dateutil
from scipy.sparse import lil_matrix # To build sparse feature matrices, if you like
from scipy.sparse import csr_matrix

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [6]:
dataset = json.load(open("train-transcripts-aligned.json", 'r+'))

In [51]:
print(dataset['ep-1'][0]['duration'])

0.4099999999999999


In [22]:
question_length = []
total_length = []
statement_length = []
data = dataset['ep-1']

for d in data:
    if d['has_q'] is True:
        question_length.append(d['duration'])
    else:
        statement_length.append(d['duration'])
    total_length.append(d['duration'])

In [27]:
# question utterance duration is about 0.775 times shorter than statement utterance duration
q_length = sum(question_length)
t_length = sum(total_length)
s_length = sum(statement_length)
print(q_length, s_length, t_length)
print(s_length/q_length)
print(t_length/q_length)

q_average = q_length/len(question_length)
s_average = s_length/len(statement_length)
t_average = t_length/len(total_length)
print('average q,s,t: ', q_average, s_average, t_average)
print(q_average/s_average)

801.5500000000005 2723.4133333333316 3524.9633333333322
3.397683654585902
4.397683654585903
average q,s,t:  15.414423076923088 19.878929440389282 18.650599647266308
0.7754151511602343


In [52]:
utterance_lengths = []
durations = []

# Iterating through each episode
for episode_id, turns in dataset.items():
    # Iterating through each turn in the episode
    for turn in turns:
        utterance_length = len(turn['utterance'])
        duration = turn['utterance_end'] - turn['utterance_start']

        utterance_lengths.append(utterance_length)
        durations.append(duration)
        
X = np.array(utterance_lengths).reshape(-1, 1)  # Features (utterance lengths)
y = np.array(durations)  # Target (durations)

print("X sample:", X[:5])  # Print first 5 samples of features
print("y sample:", y[:5])  # Print first 5 samples of targets

X sample: [[13]
 [10]
 [20]
 [38]
 [32]]
y sample: [0.41 0.81 1.45 1.61 1.44]


In [53]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the model
model = LinearRegression()
model.fit(X_train, y_train)

# Test the model (optional)
score = model.score(X_test, y_test)
print("Model accuracy:", score)

Model accuracy: 0.7533362253585961


In [58]:
utterance_lengths = []
durations = []
has_questions = []  # List to store 'has_q' feature

# Iterating through each episode
for episode_id, turns in dataset.items():
    # Iterating through each turn in the episode
    for turn in turns:
        utterance_length = len(turn['utterance'])
        duration = turn['utterance_end'] - turn['utterance_start']
        has_question = int(turn['has_q'])  # Convert boolean to int (True to 1, False to 0)

        utterance_lengths.append(utterance_length)
        durations.append(duration)
        has_questions.append(has_question)
        
X2 = np.column_stack((utterance_lengths, has_questions))  # Combine utterance lengths and has_q features
y2 = np.array(durations)  # Target (durations)

In [59]:
# Split the data into training and testing sets
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=42)

# Create and train the model
model = LinearRegression()
model.fit(X2_train, y2_train)

# Test the model (optional)
score2 = model.score(X2_test, y2_test)
print("Model accuracy:", score2)

Model accuracy: 0.7533167433119695
